# Clasificador BackPropagattion



Importar las librerías necesarias

In [38]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.metrics import Accuracy
from keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image
import numpy as np



Preprocesamiento de las imagenes

In [39]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_datagen.flow_from_directory(
    'imagenes',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    'imagenes_prueba',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


Found 2840 images belonging to 5 classes.


Found 387 images belonging to 5 classes.


Construccion del modelo

In [40]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=5, activation='softmax'))  # 5 clases: autobus, avion, bicicletas, botes, sofa


Compilacion del modelo

In [41]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=[Accuracy()])


Entrenamiento del modelo

In [42]:
history = model.fit(
    train_set,
    steps_per_epoch=len(train_set),
    epochs=8,
    validation_data=test_set,
    validation_steps=len(test_set)
)


Epoch 1/8
89/89 [==============================] - 7s 76ms/step - loss: 1.3192 - accuracy: 0.0000e+00 - val_loss: 1.1622 - val_accuracy: 0.0000e+00
Epoch 2/8
89/89 [==============================] - 7s 74ms/step - loss: 1.0704 - accuracy: 0.0000e+00 - val_loss: 1.0311 - val_accuracy: 0.0000e+00
Epoch 3/8
89/89 [==============================] - 7s 74ms/step - loss: 0.9578 - accuracy: 0.0000e+00 - val_loss: 0.9735 - val_accuracy: 0.0000e+00
Epoch 4/8
89/89 [==============================] - 7s 73ms/step - loss: 0.8877 - accuracy: 0.0000e+00 - val_loss: 0.9869 - val_accuracy: 0.0000e+00
Epoch 5/8
89/89 [==============================] - 7s 74ms/step - loss: 0.8010 - accuracy: 0.0000e+00 - val_loss: 0.8569 - val_accuracy: 0.0000e+00
Epoch 6/8
89/89 [==============================] - 7s 74ms/step - loss: 0.7472 - accuracy: 0.0000e+00 - val_loss: 0.8895 - val_accuracy: 0.0000e+00
Epoch 7/8
89/89 [==============================] - 7s 75ms/step - loss: 0.6878 - accuracy: 0.0000e+00 - val_loss

In [43]:
# Evaluar el modelo en el conjunto de prueba
eval_result = model.evaluate(test_set)
print(f"Test loss: {eval_result[0]}, Test accuracy: {eval_result[1]}")

# Si deseas ver el historial de entrenamiento para observar cómo evolucionan las métricas
print(history.history)


13/13 [==============================] - 1s 50ms/step - loss: 0.7078 - accuracy: 0.0000e+00
Test loss: 0.7077938914299011, Test accuracy: 0.0
{'loss': [1.3192282915115356, 1.0703822374343872, 0.9577572345733643, 0.8877211809158325, 0.8009858727455139, 0.7471962571144104, 0.6877686381340027, 0.6409595608711243], 'accuracy': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'val_loss': [1.1622215509414673, 1.0310992002487183, 0.9734637141227722, 0.9869213104248047, 0.8569303750991821, 0.8894884586334229, 0.7773459553718567, 0.7077938914299011], 'val_accuracy': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}


In [44]:
img_path = "imagenes_prueba/autobus/2008_004968.jpg"

In [45]:
img = image.load_img(img_path, target_size=(64, 64))

# Convertir la imagen a un array y expandir las dimensiones para que se ajuste al modelo
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)

 
img_batch /= 255.

# Hacer la predicción
prediction = model.predict(img_batch)
prediction_label = np.argmax(prediction)

# Traducir la etiqueta numérica a una palabra clave de acuerdo a tu conjunto de datos
labels = train_set.class_indices
labels = dict((v, k) for k, v in labels.items())
predicted_label = labels[prediction_label]

print(f"El modelo predice que la imagen es un(a) {predicted_label}")


1/1 [==============================] - 0s 44ms/step
El modelo predice que la imagen es un(a) autobus


Metricas

In [46]:
img = image.load_img(img_path, target_size=(64, 64))

# Convertir la imagen a un array y expandir las dimensiones para que se ajuste al modelo
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

In [47]:
predictions = model.predict(img_array)

1/1 [==============================] - 0s 17ms/step


In [48]:
predicted_index = np.argmax(predictions[0])
predicted_label = labels[predicted_index]  # Suponiendo que 'labels' es una lista de las etiquetas de tus clases
predicted_probability = predictions[0][predicted_index]

In [49]:
print(f"Etiqueta predicha: {predicted_label}, Probabilidad: {predicted_probability * 100:.2f}%")


Etiqueta predicha: autobus, Probabilidad: 100.00%
